<a href="https://colab.research.google.com/github/miNts3524/Bioinformatics/blob/main/Gromacs_mdrun.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@markdown In the following cells, applications are downloaded from a **persistent cache** in your Google Drive.
#@markdown
#@markdown This cell sets up the cache folder.
import os
if not os.path.isdir("/content/drive/MyDrive"):
  from google.colab import drive
  drive.mount("/content/drive")

if not os.path.isdir("/content/drive/MyDrive"):
  raise RuntimeError("Error: could not connect to Google Drive")
storage = "/content/drive/MyDrive/gromacs-on-colab"
%env STORAGE={storage}

Mounted at /content/drive
env: STORAGE=/content/drive/MyDrive/gromacs-on-colab


In [ ]:
%%bash
#@markdown **GROMACS** is installed from cache.

if [[ -d "/usr/local/gromacs" ]]; then
  exit 0  # already installed
fi

gromacs_vers="2024.2" #@param {type: "string"}
cache_gromacs="${STORAGE}/gromacs-${gromacs_vers}.tar.gz"

if [[ -s "${cache_gromacs}" ]]; then
  tar -xzf "${cache_gromacs}" -C "/usr/local"
else
  echo "Error: GROMACS installation not found" >&2
  echo "(Have you installed GROMACS to your Google Drive?)" >&2
  exit 1
fi

In [ ]:
#@title Set project directory & File name
aname = []
alist = []


def change(key, value):
  global aname
  global alist
  alist.append(key)
  aname.extend(value)

#@markdown #For single MD run
single_project = False #@param {type : "boolean"}
project_path="/content/drive/MyDrive/GROMACS/binder_335" #@param P{type : "string"}

change("project_path", project_path)

deff_name="Trka_binder" #@param {type : "string"}

change("deff_name", deff_name)

#@markdown <hr />
#@markdown  <br /> <br />

#@markdown  #For multi - MD run
multi_project = True #@param {type : "boolean"}
project_mother_path = '/content/drive/MyDrive/GROMACS/INSR_test' # @param {"type":"string"}
#@markdown   _File default name must be same with Folder name_
#@markdown <hr />


if single_project == multi_project:
  print("Single project or Multi project? You should check")
  raise ValueError("Single project or Multi project? You should check")
else:
  print("Single project : ", single_project)
  print("Multi project : ", multi_project)
if single_project:
  if not os.path.exists(project_path):
    print("Project path does not exist")
    raise ValueError("Project path does not exist")
  else:
    print("Project path : ", project_path)
    print("Deff name : ", deff_name)
elif multi_project:
  if not os.path.exists(project_mother_path):
    print("Project mother path does not exist")
    raise ValueError("Project mother path does not exist")
  else:
    print("Project mother path : ", project_mother_path)

# set MD run function
import os
def single_md(project_path, deff_name):
  md_mdp = "md.mdp"

  os.chdir(project_path)

  #MD run
  ! source /usr/local/gromacs/bin/GMXRC && gmx grompp -f {md_mdp} -c {deff_name}_npt.gro -p {deff_name}_topol.top -o {deff_name}_md.tpr
  ! source /usr/local/gromacs/bin/GMXRC && gmx mdrun -v -deffnm {deff_name}_md

  # Remove PBC effects for MM/PB(GB)SA
  ! source /usr/local/gromacs/bin/GMXRC; echo 0 | gmx trjconv -s {deff_name}_md.tpr -f {deff_name}_md.xtc -o {deff_name}_md_pbc_whole.xtc -pbc whole
  ! source /usr/local/gromacs/bin/GMXRC; echo 0 | gmx trjconv -s {deff_name}_md.tpr -f {deff_name}_md_pbc_whole.xtc -o {deff_name}_md_pbc_nojump.xtc -pbc nojump

  ! source /usr/local/gromacs/bin/GMXRC && \
    echo -e "splitch 1 \n name 17 Ag \n name 18 Ab \n q" | gmx make_ndx -f {deff_name}_md.tpr -o index.ndx

  ! source /usr/local/gromacs/bin/GMXRC && \
    echo -e "4 \n 4\n" | gmx rms -s {deff_name}_md.tpr -f {deff_name}_md_pbc_nojump.xtc -o rmsd_complex.xvg -n index.ndx

  ! source /usr/local/gromacs/bin/GMXRC && \
    echo -e "Ag \n Ab \n" | gmx hbond -s {deff_name}_md.tpr -f {deff_name}_md_pbc_nojump.xtc -num hb_A_B_num.xvg -n index.ndx

def single_md_mk(project_path, deff_name):
  md_mdp = os.path.join(project_path, "states_mdp","md.mdp")
  graph_path = os.path.join(project_path, "graph")
  project_path = os.path.join(project_path, "etcetera")
  if not os.path.exists(graph_path):
    os.makedirs(graph_path)
  os.chdir(project_path)

  rmsd_path = os.path.join(graph_path, "rmsd_complex.xvg")
  hb_path = os.path.join(graph_path, "hb_A_B_num.xvg")

  #MD run
  ! source /usr/local/gromacs/bin/GMXRC && gmx grompp -f {md_mdp} -c {deff_name}_npt.gro -p {deff_name}_topol.top -o {deff_name}_md.tpr
  ! source /usr/local/gromacs/bin/GMXRC && gmx mdrun -v -deffnm {deff_name}_md

  # Remove PBC effects for MM/PB(GB)SA
  ! source /usr/local/gromacs/bin/GMXRC; echo 0 | gmx trjconv -s {deff_name}_md.tpr -f {deff_name}_md.xtc -o {deff_name}_md_pbc_whole.xtc -pbc whole
  ! source /usr/local/gromacs/bin/GMXRC; echo 0 | gmx trjconv -s {deff_name}_md.tpr -f {deff_name}_md_pbc_whole.xtc -o {deff_name}_md_pbc_nojump.xtc -pbc nojump

  ! source /usr/local/gromacs/bin/GMXRC && \
    echo -e "splitch 1 \n name 17 Ag \n name 18 Ab \n q" | gmx make_ndx -f {deff_name}_md.tpr -o index.ndx

  ! source /usr/local/gromacs/bin/GMXRC && \
    echo -e "4 \n 4\n" | gmx rms -s {deff_name}_md.tpr -f {deff_name}_md_pbc_nojump.xtc -o {rmsd_path} -n index.ndx

  ! source /usr/local/gromacs/bin/GMXRC && \
    echo -e "Ag \n Ab \n" | gmx hbond -s {deff_name}_md.tpr -f {deff_name}_md_pbc_nojump.xtc -num {hb_path}.xvg -n index.ndx



Single project :  False
Multi project :  True
Project mother path :  /content/drive/MyDrive/GROMACS/INSR_test


In [ ]:
#@title Run Single MD simulations and Analyses

import os
if single_project:
  single_md(project_path, deff_name)
else:
  print("You didn't checked single - MD simulations")

You didn't checked single - MD simulations


<hr />


In [ ]:
#@title Run multi - MD simulations

import os
if multi_project:
  project_folders = os.listdir(project_mother_path)
  total_len = len(project_folders)
  print(f"Total number of folders: {total_len}")
  count = 1
  for project_folder in project_folders:
    project_path = os.path.join(project_mother_path, project_folder)
    deff_name = project_folder
    print(f"MD run for : {project_folder} in {project_path}")
    print(f"Processing {count}/{total_len}")
    single_md_mk(project_path, deff_name)
    count += 1
else:
  print("You didn't checked multi - MD simulations")

Total number of folders: 6
MD run for : binder_95 in /content/drive/MyDrive/GROMACS/INSR_test/binder_95
Processing 1/6
                      :-) GROMACS - gmx grompp, 2024.2 (-:

Executable:   /usr/local/gromacs/bin/gmx
Data prefix:  /usr/local/gromacs
Working dir:  /content/drive/MyDrive/GROMACS/INSR_test/binder_95/etcetera
Command line:
  gmx grompp -f /content/drive/MyDrive/GROMACS/INSR_test/binder_95/states_mdp/md.mdp -c binder_95_npt.gro -p binder_95_topol.top -o binder_95_md.tpr

Ignoring obsolete mdp entry 'ns_type'
Setting the LD random seed to 1869033274

Generated 2145 of the 2145 non-bonded parameter combinations
Generating 1-4 interactions: fudge = 0.5

Generated 2145 of the 2145 1-4 parameter combinations

Excluding 3 bonded neighbours molecule type 'Protein_chain_A'

turning H bonds into constraints...

Excluding 3 bonded neighbours molecule type 'Protein_chain_B'

turning H bonds into constraints...

Excluding 2 bonded neighbours molecule type 'SOL'

turning H bonds into

In [ ]:
#@title Charmm-GUI input generation







In [ ]:
#@markdown Finally, disconnect the runtime.
disconnect = True #@param {type: "boolean"}

if disconnect:
  from google.colab import drive, runtime
  drive.flush_and_unmount()
  runtime.unassign()